In [ ]:
pip install gradio diffusers transformers accelerate opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [ ]:
import gradio as gr
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel
from diffusers.utils import load_image
import torch, cv2
import numpy as np
from PIL import Image

from huggingface_hub import login
login("Hugging_face TOKEN")  # Replace with your token

# 1. Canny Edge Generator
def apply_canny(image):
    image = np.array(image)
    edges = cv2.Canny(image, 100, 200)
    edges = edges[:, :, None]
    edges = np.concatenate([edges] * 3, axis=2)
    return Image.fromarray(edges)

# 2. Load ControlNet and Pipeline
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

# 3. Redesign Logic
def redesign(image, prompt):
    image = image.convert("RGB").resize((512, 512))
    canny = apply_canny(image)

    enhanced_prompt = f"{prompt}, intricate detail, painted texture, vivid colors, symmetrical pattern, 4K product design"

    result = pipe(
        prompt=enhanced_prompt,
        image=image,
        control_image=canny,
        strength=0.7,  # allow more creativity
        guidance_scale=12.0,  # more focus on prompt
        num_inference_steps=40
    )
    return result.images[0]

# 4. Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🖌️ Redesign Your Object (Preserve Shape, Add Details)")
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload your object")
            prompt_input = gr.Textbox(label="Describe your redesign idea")
            submit_btn = gr.Button("Submit")
        with gr.Column():
            output_image = gr.Image(label="Redesigned Image")

    submit_btn.click(fn=redesign, inputs=[image_input, prompt_input], outputs=output_image)

demo.launch(debug=True, share=True)


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]